# Projeto Branch & Bound
Implementação do algoritmo Branch & Bound para programação linear inteira.

Erlon Lacerda Avelino, 20220071286  
Maria Eduarda Bandeira, 20220007230

[link do vídeo](https://youtu.be/BIlfqQGWzlc)

In [17]:
from mip import (CBC, CONTINUOUS, MAXIMIZE, Model, xsum, OptimizationStatus)
import numpy as np
from pandas import DataFrame

In [18]:
def readFile(filename: str):
    FILE = open(filename, "r")
    lines = FILE.readlines()
    
    # separa os arrays em '\n'
    lines = [lines.split() for lines in lines]

    FILE.close()
    # número de variáveis
    # número de restrições
    num_vars, num_res = int(lines[0][0]), int(lines[0][1])

    lines = lines[1:]

    # coeficientes da função objetivo
    coef_fo = lines[0]
    coef_fo = [float(i) for i in coef_fo]

    # coeficientes das restrições
    coef_restantes = lines[1:]
    coef_restantes = [[float(i) for i in coef_restantes[j]] for j in range(num_res)]

    coef_left = [coef_restantes[j][:-1] for j in range(num_res)]

    coef_right = [coef_restantes[j][-1] for j in range(num_res)]


    m = Model(sense=MAXIMIZE, solver_name=CBC)

    # variáveis
    x = [m.add_var(var_type=CONTINUOUS, lb=0.0, ub=1.0, name="x_"+str(i)) for i in range(num_vars)]

    # função objetivo
    m.objective = xsum(coef_fo[i] * x[i] for i in range(num_vars))

      #cria as restrições
      #model += xsum(coefientes[i]*x[i] for i in range(num_vars)) <= coefientes[-1] 

    # restrições
    for i in range(num_res):
        m += xsum(coef_left[i][j] * x[j] for j in range(num_vars)) <= coef_right[i]
    # 'silencia' o solver
    m.verbose = 0

    return m

In [19]:
class No:
    def __init__ (self, modelo : Model):
        self.modelo = modelo
        self.modelo.optimize()
        self.Z = self.modelo.objective_value

In [20]:
def poda_integralidade (modelo: Model):
    # Se todas as variáveis são inteiras, o nó é viável
    # logo, deve ser podado
    for var in modelo.vars:
        if var.x != int(var.x):
            return False
    return True
# poda por limitante, analisa o valor da função objetivo
def poda_limitante (lim_inf, Z):
    # Se o valor da função objetivo for menor ou igual ao limitante inferior,
    # qualquer nó abaixo incluindo o próprio nó é descartado,
    # pois já há uma solução melhor encontrada
    if Z <= lim_inf:
        return True
    else:
        return False

# poda por inviabilidade, analisa o status do modelo
def poda_inviabilidade (modelo: Model):
    # Se o modelo não for ótimo, ele é inviável
    # ou não foi encontrada uma solução
    if not modelo.status == OptimizationStatus.OPTIMAL:
        return True

    # Inviável
    #if modelo.status == OptimizationStatus.INFEASIBLE:
    #    return True
    
    # Não encontrou solução
    #elif modelo.status == OptimizationStatus.NO_SOLUTION_FOUND:
    #    return True
    return False

In [21]:
def print_no(otimo: Model):
    print(f"Valor da solução  = {otimo.objective_value:.2f}")

    print("Solução:")
    for v in otimo.vars:
        print(f"{v.name} = {v.x:.2f}")

In [22]:
def closest_value(array, value): 
    array = np.asarray(array)
    
    value_found = np.absolute(array - value)
    value_found = value_found.argmin()
    
    return value_found 

In [23]:
# Gera os dois nós filhos do algoritmo a partir da variável mais próxima de 0.5
def branch (pai):
    # escolhe a variável mais próxima de 0.5
    modelo_vars = [v.x for v in pai.modelo.vars]
    
    var_escolhida = closest_value(modelo_vars, 0.5)
    
    # cria os modelos filhos
    # e adiciona restrição para a variável escolhida
    m1 = pai.modelo.copy()
    m1.verbose = 0 # 'silencia' o solver
    m1 += m1.vars[var_escolhida] == 0

    m2 = pai.modelo.copy()
    m2.verbose = 0
    m2 += m2.vars[var_escolhida] == 1

    # cria os nós filhos
    filho1 = No(m1)
    filho2 = No(m2)

    return filho1, filho2


In [24]:
class Branch_and_Bound:
    def __init__ (self, no_modelo: No):
        self.no_modelo = no_modelo

    def executar (self):
        lim_inf = float('-inf')
        fila = [self.no_modelo]
        solucao = "Nenhuma solução encontrada."
        
        while fila:
            no_atual = fila.pop(0)

            if poda_inviabilidade(no_atual.modelo):
                continue
            
            if poda_limitante(lim_inf, no_atual.Z):
                continue
            
            if poda_integralidade(no_atual.modelo):
                if no_atual.Z > lim_inf:
                    lim_inf = no_atual.Z
                    solucao  = no_atual.modelo
                continue


            filho1, filho2 = branch(no_atual)
            fila.append(filho1)
            fila.append(filho2)

        print_no(solucao)

In [25]:
modelo = readFile('teste.txt')
teste = No(modelo)

BnB = Branch_and_Bound(teste)
BnB.executar()

Valor da solução  = 10.00
Solução:
x_0 = 0.00
x_1 = 1.00
x_2 = 0.00


In [11]:
modelo1 = readFile('testes/teste1.txt')
teste1 = No(modelo1)
BnB1 = Branch_and_Bound(teste1)
BnB1.executar()

Valor da solução  = 20.00
Solução:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 1.00
x_5 = 1.00
x_6 = 0.00


In [12]:
modelo2 = readFile('testes/teste2.txt')
teste2 = No(modelo2)
BnB2 = Branch_and_Bound(teste2)
BnB2.executar()

Valor da solução  = 24.00
Solução:
x_0 = 0.00
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 1.00
x_6 = 1.00
x_7 = 0.00
x_8 = 1.00


In [13]:
modelo3 = readFile('testes/teste3.txt')
teste3 = No(modelo3)
BnB3 = Branch_and_Bound(teste3)
BnB3.executar()

Valor da solução  = 19.00
Solução:
x_0 = 0.00
x_1 = 0.00
x_2 = 1.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 1.00
x_7 = 0.00
x_8 = 0.00


In [14]:
modelo4 = readFile('testes/teste4.txt')
teste4 = No(modelo4)
BnB4 = Branch_and_Bound(teste4)
BnB4.executar()

Valor da solução  = 10.00
Solução:
x_0 = 0.00
x_1 = 0.00
x_2 = 1.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.00
